In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max.rows', 150)
pd.set_option('display.max.columns', 150)

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error

import xgboost as xgb
from xgboost import XGBRegressor

from hyperopt import fmin, hp, tpe, Trials, space_eval, STATUS_OK, STATUS_RUNNING
from functools import partial
from tqdm import tqdm_notebook as tqdm

from category_encoders import OrdinalEncoder, OneHotEncoder
# import eli5
# from eli5.sklearn import PermutationImportance

from scipy.stats import randint, uniform

In [2]:
ATOMIC_NUMBERS = {
    'H': 1,
    'C': 6,
    'N': 7,
    'O': 8,
    'F': 9
}

In [3]:
trainval = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
structures = pd.read_csv('./data/structures.csv')
dipole = pd.read_csv('./data/dipole_moments.csv')
contrib = pd.read_csv('./data/scalar_coupling_contributions.csv')
magnetic = pd.read_csv('./data/magnetic_shielding_tensors.csv')
mulliken = pd.read_csv('./data/mulliken_charges.csv')
potential_energy = pd.read_csv('./data/potential_energy.csv')
# test_mulliken = pd.read_csv('mulliken_charges_test_set.csv')

## Reduce Memory Function

In [4]:
def reduce_mem_usage(df, verbose=True):
    """
    This function reduces the numeric to the least possible numeric type that fits the data so 
    memory usage during transforming and training will be reduced.
    Taken from: https://www.kaggle.com/todnewman/keras-neural-net-for-champs
    
    Han
    Parameters:
    ===========
    dataframe: input dataframe 
    verbose: verbose mode, default True.
    Output:
    ===========
    dataframe: dataframe with numeric columns types changed to the least possible size
    """

    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [5]:
train_dtypes = {
    'molecule_name': 'category',
    'atom_index_0': 'int8',
    'atom_index_1': 'int8',
    'type': 'category',
    'scalar_coupling_constant': 'float32'
}

In [6]:
trainval['molecule_index'] = trainval.molecule_name.str.replace('dsgdb9nsd_', '').astype('int32')
train_csv = trainval[['molecule_index', 'atom_index_0', 'atom_index_1', 'type', 'scalar_coupling_constant']]
train_csv.head(10)

,molecule_index,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,1,1,0,1JHC,84.8076
1,1,1,2,2JHH,-11.2570
2,1,1,3,2JHH,-11.2548
3,1,1,4,2JHH,-11.2543
4,1,2,0,1JHC,84.8074
5,1,2,3,2JHH,-11.2541
6,1,2,4,2JHH,-11.2548
7,1,3,0,1JHC,84.8093
8,1,3,4,2JHH,-11.2543
9,1,4,0,1JHC,84.8095


In [7]:
print('Shape: ', train_csv.shape)
print('Total: ', train_csv.memory_usage().sum())
train_csv.memory_usage()

Shape:  (4658147, 5)
Total:  167693372


Index                             80
molecule_index              18632588
atom_index_0                37265176
atom_index_1                37265176
type                        37265176
scalar_coupling_constant    37265176
dtype: int64

In [8]:
test['molecule_index'] = test['molecule_name'].str.replace('dsgdb9nsd_', '').astype('int32')
test_csv = test[['molecule_index', 'atom_index_0', 'atom_index_1', 'type']]
test_csv.head(10)

,molecule_index,atom_index_0,atom_index_1,type
0,4,2,0,2JHC
1,4,2,1,1JHC
2,4,2,3,3JHH
3,4,3,0,1JHC
4,4,3,1,2JHC
5,15,3,0,1JHC
6,15,3,2,3JHC
7,15,3,4,2JHH
8,15,3,5,2JHH
9,15,4,0,1JHC


In [9]:
structures_dtypes = {
    'molecule_name': 'category',
    'atom_index': 'int8',
    'atom': 'category',
    'x': 'float32',
    'y': 'float32',
    'z': 'float32'
}
structures['molecule_index'] = structures.molecule_name.str.replace('dsgdb9nsd_', '').astype('int32')
structures_csv = structures[['molecule_index', 'atom_index', 'atom', 'x', 'y', 'z']]
structures_csv['atom'] = structures_csv['atom'].replace(ATOMIC_NUMBERS).astype('int8')
structures_csv.head(10)

/home/user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


,molecule_index,atom_index,atom,x,y,z
0,1,0,6,-0.012698,1.085804,0.008001
1,1,1,1,0.002150,-0.006031,0.001976
2,1,2,1,1.011731,1.463751,0.000277
3,1,3,1,-0.540815,1.447527,-0.876644
4,1,4,1,-0.523814,1.437933,0.906397
5,2,0,7,-0.040426,1.024108,0.062564
6,2,1,1,0.017257,0.012545,-0.027377
7,2,2,1,0.915789,1.358745,-0.028758
8,2,3,1,-0.520278,1.343532,-0.775543
9,3,0,8,-0.034360,0.977540,0.007602


## Build Distance Dataset

In [10]:
def build_type_dataframes(base, structures, coupling_type):
    base = base[base['type'] == coupling_type].drop('type', axis=1).copy()
    base = base.reset_index()
#     base['id'] = base['id'].astype('int32')
    structures = structures[structures['molecule_index'].isin(base['molecule_index'])]
    return base, structures

In [11]:
def add_coordinates(base, structures, index):
    df = pd.merge(base, structures, how='inner',
                  left_on=['molecule_index', f'atom_index_{index}'],
                  right_on=['molecule_index', 'atom_index']).drop(['atom_index'], axis=1)
    df = df.rename(columns={
        'atom': f'atom_{index}',
        'x': f'x_{index}',
        'y': f'y_{index}',
        'z': f'z_{index}'
    })
    return df

In [12]:
def add_atoms(base, atoms):
    df = pd.merge(base, atoms, how='inner',
                  on=['molecule_index', 'atom_index_0', 'atom_index_1'])
    return df

In [13]:
def merge_all_atoms(base, structures):
    df = pd.merge(base, structures, how='left',
                  left_on=['molecule_index'],
                  right_on=['molecule_index'])
    df = df[(df.atom_index_0 != df.atom_index) & (df.atom_index_1 != df.atom_index)]
    return df

In [14]:
def add_center(df):
    df['x_c'] = ((df['x_1'] + df['x_0']) * np.float32(0.5))
    df['y_c'] = ((df['y_1'] + df['y_0']) * np.float32(0.5))
    df['z_c'] = ((df['z_1'] + df['z_0']) * np.float32(0.5))

def add_distance_to_center(df):
    df['d_c'] = ((
        (df['x_c'] - df['x'])**np.float32(2) +
        (df['y_c'] - df['y'])**np.float32(2) + 
        (df['z_c'] - df['z'])**np.float32(2)
    )**np.float32(0.5))

def add_distance_between(df, suffix1, suffix2):
    df[f'd_{suffix1}_{suffix2}'] = ((
        (df[f'x_{suffix1}'] - df[f'x_{suffix2}'])**np.float32(2) +
        (df[f'y_{suffix1}'] - df[f'y_{suffix2}'])**np.float32(2) + 
        (df[f'z_{suffix1}'] - df[f'z_{suffix2}'])**np.float32(2)
    )**np.float32(0.5))

In [15]:
def add_distances(df):
    n_atoms = 1 + max([int(c.split('_')[1]) for c in df.columns if c.startswith('x_')])
    
    for i in range(1, n_atoms):
        for vi in range(min(4, i)):
            add_distance_between(df, i, vi)

In [16]:
def add_n_atoms(base, structures):
    dfs = structures['molecule_index'].value_counts().rename('n_atoms').to_frame()
    return pd.merge(base, dfs, left_on='molecule_index', right_index=True)

In [25]:
def build_couple_dataframe(some_csv, structures_csv, coupling_type, n_atoms=10):
    base, structures = build_type_dataframes(some_csv, structures_csv, coupling_type)
    base = add_coordinates(base, structures, 0)
    base = add_coordinates(base, structures, 1)
    
    base = base.drop(['atom_0', 'atom_1'], axis=1)
#     print('>>>>', base.head())   
#     atoms = base.drop('id', axis=1).copy()
    atoms = base.drop('index', axis=1).copy()
    if 'scalar_coupling_constant' in some_csv:
        atoms = atoms.drop(['scalar_coupling_constant'], axis=1)
        
    add_center(atoms)
    atoms = atoms.drop(['x_0', 'y_0', 'z_0', 'x_1', 'y_1', 'z_1'], axis=1)

    atoms = merge_all_atoms(atoms, structures)
    
    add_distance_to_center(atoms)
    
    atoms = atoms.drop(['x_c', 'y_c', 'z_c', 'atom_index'], axis=1)
    atoms.sort_values(['molecule_index', 'atom_index_0', 'atom_index_1', 'd_c'], inplace=True)
    atom_groups = atoms.groupby(['molecule_index', 'atom_index_0', 'atom_index_1'])
    atoms['num'] = atom_groups.cumcount() + 2
    atoms = atoms.drop(['d_c'], axis=1)
    atoms = atoms[atoms['num'] < n_atoms]

    atoms = atoms.set_index(['molecule_index', 'atom_index_0', 'atom_index_1', 'num']).unstack()
    atoms.columns = [f'{col[0]}_{col[1]}' for col in atoms.columns]
    atoms = atoms.reset_index()
    
    # downcast back to int8
    for col in atoms.columns:
        if col.startswith('atom_'):
            atoms[col] = atoms[col].fillna(0).astype('int8')
            
    atoms['molecule_index'] = atoms['molecule_index'].astype('int32')
    
    full = add_atoms(base, atoms)
    add_distances(full)
    
#     print(full.head())
#     full.sort_values('id', inplace=True)
    full.sort_values('index', inplace=True)
    
    return full

In [18]:
def take_n_atoms(df, n_atoms, four_start=4):
    labels = []
    for i in range(2, n_atoms):
        label = f'atom_{i}'
        labels.append(label)

    for i in range(n_atoms):
        num = min(i, 4) if i < four_start else 4
        for j in range(num):
            labels.append(f'd_{i}_{j}')
    if 'scalar_coupling_constant' in df:
        labels.append('scalar_coupling_constant')
    return df[labels]

In [19]:
def build_x_y_data(some_csv, coupling_type, n_atoms):
    full = build_couple_dataframe(some_csv, structures_csv, coupling_type, n_atoms=n_atoms)
    
    df = take_n_atoms(full, n_atoms)
    df = df.fillna(0)
    print(df.columns)
    
#     if 'scalar_coupling_constant' in df:
#         X_data = df.drop(['scalar_coupling_constant'], axis=1).values.astype('float32')
#         y_data = df['scalar_coupling_constant'].values.astype('float32')
#     else:
#         X_data = df.values.astype('float32')
#         y_data = None
        
    if 'scalar_coupling_constant' in df:
        X_data = df.drop(['scalar_coupling_constant'], axis=1)  #.values.astype('float32')
        y_data = df['scalar_coupling_constant']  #.values.astype('float32')
    else:
        X_data = df  #.values.astype('float32')
        y_data = None
    
    return X_data, y_data #  full.molecule_index.values

In [20]:
model_params = {
    '1JHN': 7,
    '1JHC': 10,
    '2JHH': 9,
    '2JHN': 9,
    '2JHC': 9,
    '3JHH': 9,
    '3JHC': 10,
    '3JHN': 10
}

In [21]:
coupling_type = '1JHN'

In [23]:
n_atoms=model_params[coupling_type]
# X_data, y_data = build_x_y_data(train_csv, coupling_type, n_atoms)

In [ ]:
# X_data.head(10)

In [26]:
full = build_couple_dataframe(train_csv, structures_csv, '1JHN', n_atoms=n_atoms)
print(full.shape)

(43363, 49)


In [29]:
full.head(10)

,index,molecule_index,atom_index_0,atom_index_1,scalar_coupling_constant,x_0,y_0,z_0,x_1,y_1,z_1,atom_2,atom_3,atom_4,atom_5,atom_6,x_2,x_3,x_4,x_5,x_6,y_2,y_3,y_4,y_5,y_6,z_2,z_3,z_4,z_5,z_6,d_1_0,d_2_0,d_2_1,d_3_0,d_3_1,d_3_2,d_4_0,d_4_1,d_4_2,d_4_3,d_5_0,d_5_1,d_5_2,d_5_3,d_6_0,d_6_1,d_6_2,d_6_3
0,10,2,1,0,32.6889,0.017257,0.012545,-0.027377,-0.040426,1.024108,0.062564,1,1,0,0,0,0.915789,-0.520278,NaN,NaN,NaN,1.358745,1.343532,NaN,NaN,NaN,-0.028758,-0.775543,NaN,NaN,NaN,1.017190,1.618523,1.017187,1.618710,1.017208,1.618706,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13,2,2,0,32.6891,0.915789,1.358745,-0.028758,-0.040426,1.024108,0.062564,1,1,0,0,0,0.017257,-0.520278,NaN,NaN,NaN,0.012545,1.343532,NaN,NaN,NaN,-0.027377,-0.775543,NaN,NaN,NaN,1.017187,1.618523,1.017190,1.618706,1.017208,1.618710,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15,2,3,0,32.6905,-0.520278,1.343532,-0.775543,-0.040426,1.024108,0.062564,1,1,0,0,0,0.915789,0.017257,NaN,NaN,NaN,1.358745,0.012545,NaN,NaN,NaN,-0.028758,-0.027377,NaN,NaN,NaN,1.017208,1.618706,1.017187,1.618710,1.017190,1.618523,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,97,12,3,0,55.5252,0.825355,1.885049,0.003738,-0.025900,1.346146,0.008894,1,6,8,1,0,-0.908377,0.046467,1.071835,-0.961441,NaN,1.826796,-0.011743,-0.652588,-0.475004,NaN,0.018920,0.001204,-0.011133,0.008074,NaN,1.007511,1.734777,1.004933,2.050487,1.359838,2.071779,2.549623,2.280429,3.173246,1.209220,2.960154,2.047394,2.302437,1.109295,NaN,NaN,NaN,NaN
4,101,12,4,0,54.7359,-0.908377,1.826796,0.018920,-0.025900,1.346146,0.008894,1,6,1,8,0,0.825355,0.046467,-0.961441,1.071835,NaN,1.885049,-0.011743,-0.475004,-0.652588,NaN,0.003738,0.001204,0.008074,-0.011133,NaN,1.004933,1.734777,1.007511,2.071779,1.359838,2.050487,2.302437,2.047394,2.960154,1.109295,3.173246,2.280429,2.549623,1.209220,NaN,NaN,NaN,NaN
5,225,19,7,2,54.0640,1.984051,-0.131759,0.386749,1.178267,-0.633243,0.056917,1,6,6,1,8,1.190494,-0.027404,0.006799,0.913903,-1.037220,-1.639196,-0.017521,1.499697,1.929238,-0.639806,0.100062,-0.148948,-0.025670,-0.461001,-0.407276,1.004771,1.727509,1.006952,2.084700,1.369356,2.043309,2.596395,2.434870,3.357021,1.522602,2.472166,2.627629,3.622846,2.184789,3.164912,2.263603,2.493768,1.213961
6,229,19,8,2,56.1860,1.190494,-1.639196,0.100062,1.178267,-0.633243,0.056917,1,6,8,6,1,1.984051,-0.027404,-1.037220,0.006799,0.913903,-0.131759,-0.017521,-0.639806,1.499697,1.929238,0.386749,-0.148948,-0.407276,-0.025670,-0.461001,1.006952,1.727509,1.004771,2.043309,1.369356,2.084700,2.493768,2.263603,3.164912,1.213961,3.357021,2.434870,2.596395,1.522602,3.622846,2.627629,2.472166,2.184789
7,389,32,4,0,37.7190,0.917298,1.835475,0.052027,-0.036081,1.487625,0.036781,1,6,1,1,6,-0.471622,-0.010395,0.528272,0.489462,-1.371810,1.856772,0.025176,-0.345333,-0.408804,-0.521089,-0.802756,-0.015066,0.864695,-0.897152,0.027393,1.014969,1.631013,1.015277,2.035264,1.463593,2.046443,2.359597,2.088941,2.937592,1.096093,2.474020,2.178272,2.462809,1.102846,3.285425,2.412300,2.674642,1.467535
8,395,32,5,0,38.3495,-0.471622,1.856772,-0.802756,-0.036081,1.487625,0.036781,1,6,1,6,1,0.917298,-0.010395,0.489462,-1.371810,0.528272,1.835475,0.025176,-0.408804,-0.521089,-0.345333,0.052027,-0.015066,-0.897152,0.027393,0.864695,1.015277,1.631013,1.014969,2.046443,1.463593,2.035264,2.462809,2.178272,2.474020,1.102846,2.674642,2.412300,3.285425,1.467535,2.937592,2.088941,2.359597,1.096093
9,696,50,5,0,59.3010,-0.832432,1.927727,0.021230,-0.008297,1.353628,0.009960,6,6,1,1,6,1.280333,-0.003428,1.484718,-0.925516,2.122166,1.824574,-0.018369,2.883126,-0.576971,0.736989,-0.000225,0.002357,0.003543,0.008394,-0.014456,1.004448,2.115390,1.372028,2.115394,1.372027,2.245993,2.506449,2.137405,1.078110,3.260867,2.506459,2.137406,3.260868,1.078109,3.185716,2.218043,1.375401,2.255881
